# Prediction module

In [ ]:
#While predicting consider customer choices of the algorithm if he has given, otherwise pickup best performing algorithm
AlgorithmChoices = list(customer_tbl.find({'CustomerID':1},{'AlgorithmChoices':1,'_id':0}))[0]['AlgorithmChoices']
for predicted_field, algo_id in AlgorithmChoices.items():
    algo_name =  list(algo_tbl.find({'AlgorithmID':algo_id},{'AlgorithmName':1,'_id':0}))[0]['AlgorithmName']

In [1]:
import json
import requests
import pandas as pd

In [ ]:
"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', 'Accept': 'application/json'}

"""Reading test batch
"""
df = pd.read_csv('./data/test.csv', encoding="utf-8-sig")
df = df[20:25]

"""Converting Pandas Dataframe to json
"""
data = df.to_json(orient='records')

In [ ]:
"""POST <url>/predict
"""
resp = requests.post("http://127.0.0.1:5002/predict/RFClassifier", data = json.dumps(data), headers= header)

In [ ]:
resp.json()

# Training Module

testing

In [ ]:
"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', 'Accept': 'application/json'}

"""Reading test batch
"""
df = pd.read_csv('./data/training.csv', encoding="latin-1")

"""Converting Pandas Dataframe to json
"""
data = df.to_json(orient='records')
algoName = ['RFClassifier','SVC','MultinomialNB','LogisticRegression']
resp = requests.post('http://127.0.0.1:5002/train/'+algoName[2], data = json.dumps(data), headers= header)

In [ ]:
resp.json()

Rough work

In [1]:
from pymongo import MongoClient
import warnings
warnings.filterwarnings("ignore")
mongo = MongoClient()
db = mongo.TicketTriaging

In [2]:
customer_tbl = mongo.TicketTriaging.TblCustomer
algo_tbl = mongo.TicketTriaging.TblAlgorithm
training_tickets = mongo.TicketTriaging.TblIncidentTraining
    #get algorithm choice
#algo_id =  list(customer_tbl.find({'CustomerID':1},{'AlgorithmID':1,'_id':0}))[0]['AlgorithmID'] #for AMD

#algo_name =  list(algo_tbl.find({'AlgorithmID':algo_id},{'AlgorithmName':1,'_id':0}))[0]['AlgorithmName']

In [116]:
pred_field = 'Assignment_group'
in_field = 'Description'
training_tkt_df =  pd.DataFrame(list(training_tickets.find({'CustomerID':1},{'_id':0,in_field:1,pred_field:1})))

In [11]:
from bson import json_util

In [6]:
pred_field = 'Assignment_group'
in_field = 'Description'
a = training_tickets.find({'CustomerID':1},{'_id':0,in_field:1,pred_field:1})

In [17]:
json_util.dumps(a)

'[{"Assignment_group": "SAP BW Support"}, {"Description": "(CI=SAP HCM (Human Capital Management))Job \\"ZHCM_WW_CYC_Load_WWHRDB_Hourly_Data_To_HRETL_Database_NUS\\"  (47120848) ended with status Error", "Assignment_group": "SAP HR/PY Support"}, {"Description": "(CI=SAP BPC NW EPM (Product & Cust Eng - PCE))ATL Redwood PRD: BPC Redwood Job ZPC_CCR_INT Ended Not Ok.", "Assignment_group": "SAP BPC Support"}, {"Description": "(CI=SAP BPC NW EPM (Project Base Accounting - PBA))ATL Redwood PRD: BPC Redwood Job ZPC_PBA_MD Ended Not Ok.", "Assignment_group": "SAP BPC Support"}, {"Description": "(CI=SAP ERP FI (Finance))JC_ZFICO_B_AR_DLY_ATLUT (47110773) ended with status Error", "Assignment_group": "SAP SD Support"}, {"Description": "(CI=SAP ERP FI (Finance))ZFICO_B1_AR_DLY_PRars118D_SAP (47110775) ended with status Killed", "Assignment_group": "SAP SD Support"}, {"Description": "(CI=SAP HCM (Human Capital Management))ZHCM_OM_DLY_Remove_Roles_From_Chiefs_Automating_Chief_Roles_NUS (47118572) 

In [117]:
training_tkt_df.head()

,Assignment_group,Description
0,SAP BW Support,NaN
1,SAP HR/PY Support,"(CI=SAP HCM (Human Capital Management))Job ""ZH..."
2,SAP BPC Support,(CI=SAP BPC NW EPM (Product & Cust Eng - PCE))...
3,SAP BPC Support,(CI=SAP BPC NW EPM (Project Base Accounting - ...
4,SAP SD Support,(CI=SAP ERP FI (Finance))JC_ZFICO_B_AR_DLY_ATL...


In [118]:
training_tkt_df = training_tkt_df[pd.notnull(training_tkt_df[in_field])]

In [121]:
training_tkt_df.head()

,Assignment_group,Description,category_id
1,SAP HR/PY Support,"(CI=SAP HCM (Human Capital Management))Job ""ZH...",0
2,SAP BPC Support,(CI=SAP BPC NW EPM (Product & Cust Eng - PCE))...,1
3,SAP BPC Support,(CI=SAP BPC NW EPM (Project Base Accounting - ...,1
4,SAP SD Support,(CI=SAP ERP FI (Finance))JC_ZFICO_B_AR_DLY_ATL...,2
5,SAP SD Support,(CI=SAP ERP FI (Finance))ZFICO_B1_AR_DLY_PRars...,2


In [120]:
training_tkt_df['category_id'] = training_tkt_df[pred_field].factorize()[0]

In [4]:
algo_name

'RFClassifier'

In [123]:
list(customer_tbl.find({'CustomerID':1},{'CustomerName':1,'_id':0}))[0]['CustomerName']

'AMD'

In [103]:
for field in list(customer_tbl.find({"CustomerName" : "AMD"},{'PredictedField':1,'_id':0}))[0]['PredictedField']:
    print(field)

Assignment_group
Subcategory


In [30]:
AlgorithmChoices = list(customer_tbl.find({'CustomerID':1},{'AlgorithmChoices':1,'_id':0}))[0]['AlgorithmChoices']
for PredictedField, AlgorithmID in AlgorithmChoices.items():
    print('PredictedField:%s AlgorithmID:%s'%(PredictedField,AlgorithmID))

PredictedField:Assignment_group AlgorithmID:2
PredictedField:Subcategory AlgorithmID:1


# ------------------------------------

In [35]:
algo_listOfDicts = list(algo_tbl.find({},{'_id':0,'AlgorithmName':1}))
algo_list = []
for algo in algo_listOfDicts:
    algo_list.append(algo['AlgorithmName'])
algo_list

['LogisticRegression', 'RFClassifier', 'SVC', 'MultinomialNB']

In [36]:
[1 for i in range(10)]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Storing transaction date for every transactions

In [84]:
db.test.insert({'status':0})

ObjectId('5ba275c541e1211228ff4c1d')

In [ ]:
import pytz
from datetime import datetime

Update_many feature in PyMongo

In [86]:
db.test.update_many({'status': 0},
                   {"$set": {
                       "status": 'Completed',
                       "endtime": 'hello'}})

In [87]:
list(db.test.find())

[{'_id': ObjectId('5ba2709a41e1211228ff4c18'), 'status': 1, 'endtime': 'hey'},
 {'_id': ObjectId('5ba274c941e1211228ff4c19'), 'status': 1},
 {'_id': ObjectId('5ba274fd41e1211228ff4c1a'), 'status': 1, 'endtime': 'hey'},
 {'_id': ObjectId('5ba2750941e1211228ff4c1b'), 'status': 1, 'endtime': 'hey'},
 {'_id': ObjectId('5ba275c441e1211228ff4c1c'),
  'status': 'Completed',
  'endtime': 'hello'},
 {'_id': ObjectId('5ba275c541e1211228ff4c1d'),
  'status': 'Completed',
  'endtime': 'hello'}]

In [59]:
today = str(datetime.now(pytz.timezone('EST')))
today = today.split(' ')
''.join(today[0].split('-'))

'20180919'

Advanced formatting

# Maintaining TblTraining records

In [124]:
training_hist = db.TblTraining

In [125]:
training_hist

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'TicketTriaging'), 'TblTraining')

In [128]:
algo = 'RFClassifier'
int(list(algo_tbl.find({'AlgorithmName':algo},{'_id':0,'AlgorithmID':1}))[0]['AlgorithmID'])

2

In [131]:
accuracy = 98.23
TrainingID= 1
algo_id = int(list(algo_tbl.find({'AlgorithmName':algo},{'_id':0,'AlgorithmID':1}))[0]['AlgorithmID'])
cust_id = 1

In [132]:
training_meta = {'TrainingID':1,'AlgorithmID':algo_id,'CustomerID':cust_id,'Accuracy':accuracy}

In [133]:
training_meta

{'TrainingID': 1, 'AlgorithmID': 2, 'CustomerID': 1, 'Accuracy': 98.23}

In [134]:
training_hist.insert_one(training_meta)

In [136]:
list(training_hist.find())

[{'_id': ObjectId('5ba287c941e1211228ff4c1e'),
  'TrainingID': 1,
  'AlgorithmID': 2,
  'CustomerID': 1,
  'Accuracy': 98.23}]